Loading libraries

In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from xgboost import XGBClassifier
import xgboost as xgb

import sklearn
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, precision_recall_curve, auc, roc_curve, recall_score

/kaggle/input/metagenomics/markers2clades_DB.txt
/kaggle/input/metagenomics/abundance_stoolsubset.txt
/kaggle/input/metagenomics/abundance.txt
/kaggle/input/metagenomics/marker_presence.txt


In [21]:
# Metaparameters
VERBOSE = 0
FOLDS = 5
# show_fold_stats = True
show_fold_stats = False
# test_train_split_SEED = 1970
test_train_split_SEED = 1971

In [22]:
def plot_ROC(fpr, tpr, m_name):
    roc_auc = sklearn.metrics.auc(fpr, tpr)
    plt.figure(figsize=(6, 6))
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc, alpha=0.5)
    
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', alpha=0.5)
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.grid(True)
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.title('ROC for %s'%m_name, fontsize=20)
    plt.legend(loc="lower right", fontsize=16)
    plt.show()

In [23]:
# pd_abundance = pd.read_csv('data/abundance.txt', sep='\t', header=None, index_col=0, dtype=str)
pd_abundance = pd.read_csv('/kaggle/input/metagenomics/abundance_stoolsubset.txt', sep='\t', header=None, index_col=0, dtype=str)
ind = pd_abundance.index.tolist()
disease = pd_abundance.loc['disease',:] 
# d_name = pd_abundance.loc['dataset_name',:] 
print(disease.value_counts())
# d_name.value_counts()

diseases = ['obesity', 'cirrhosis', 't2d', 'cancer']

n                             944
t2d                           223
obesity                       164
ibd_ulcerative_colitis        148
cirrhosis                     118
leaness                        89
stec2-positive                 52
impaired_glucose_tolerance     49
cancer                         48
n_relative                     47
small_adenoma                  26
ibd_crohn_disease              25
 -                             20
large_adenoma                  13
overweight                     10
-                               7
obese                           5
underweight                     1
Name: disease, dtype: int64


In [24]:
pd_abundance = pd_abundance.transpose()
cols = pd_abundance.columns.tolist()
species = [x for x in cols if x.startswith('k_')]
descr = [x for x in cols if not x.startswith('k_')]
pd_abundance_conv = pd_abundance.copy()
pd_abundance_conv = pd_abundance_conv[species].astype('float64')
pd_abundance_conv = pd.concat([pd_abundance[descr], pd_abundance_conv], axis = 1)

data_sets = {'control':['hmp', 'hmpii'],'t2d':['WT2D','t2dmeta_long','t2dmeta_short'], 'cirrhosis' : ['Quin_gut_liver_cirrhosis'], 'cancer' : ['Zeller_fecal_colorectal_cancer'], 'obesity' : ['Chatelier_gut_obesity']}
pd_abundance_conv['disease'] = pd_abundance_conv['disease'].apply(lambda x: 'control' if ((x == 'n') or (x == 'nd') or (x == 'leaness')) else x)
disease1 = pd_abundance_conv.loc[:,'disease'] 
pd_control = pd_abundance_conv.loc[pd_abundance_conv['disease'] == 'control']
pd_disease = pd_abundance_conv.loc[pd_abundance_conv['disease'] != 'control']
not_disease = [d for d in pd_disease.disease.unique().tolist() if d not in diseases] # don't consider these diseases



In [25]:
def disease_distinction(d, counter_d):
    print('-' * 80)
    print('Discriminate %s from %s'%(d,counter_d))
    
    skf = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = SEED)
    oof_preds = []
    oof_aucs = []
    oof_scores= []
    ds_names = data_sets[d]
    if len(ds_names) == 1:
        pd_cont = pd_control.loc[pd_control['dataset_name'] == ds_names[0]]
        pd_dis = pd_disease.loc[pd_disease['dataset_name'] == ds_names[0]]
    else:
        pd_cont = pd_control.loc[pd_control['dataset_name'] == ds_names[0]]
        pd_dis = pd_disease.loc[(pd_disease['disease'] == d) & (pd_disease['dataset_name'] == ds_names[0])]
        for ds in ds_names[1:]:
            pd_cont = pd.concat([pd_cont, pd_control.loc[pd_control['dataset_name'] == ds]], axis = 0)
            pd_dis = pd.concat([pd_dis, pd_disease.loc[(pd_disease['disease'] == d) & (pd_disease['dataset_name'] == ds)]], axis = 0)
                                 
    # create dataset with all other diseases, target set to 0
    pd_others = pd_disease.loc[(pd_disease['disease'] == counter_d)]
    target_others = pd_others['disease'].apply(lambda x: 1 if x == d else 0)
    
    pd_train = pd.concat([pd_cont, pd_dis], axis = 0)
    # adding control data from healthy subject, data by HMP
    pd_train = pd.concat([pd_train, pd_control.loc[pd_control['dataset_name'] == 'hmp']], axis = 0)
    pd_train = pd.concat([pd_train, pd_control.loc[pd_control['dataset_name'] == 'hmpii']], axis = 0)
    target = pd_train['disease']
    # convert text target into binary
    binary_target = target.apply(lambda x: 1 if x == d else 0)
    # binary_target.value_counts()
    # use only abundance data for training
    pd_train  = pd_train[species] 
    disease_train, disease_test, disease_y_train, disease_y_test = train_test_split(pd_train, binary_target, test_size = 0.10, random_state = test_train_split_SEED)   
    pd_others = pd_others[species] 
    full_test = pd.concat([disease_test, pd_others])
    full_y_test = pd.concat([disease_y_test, target_others])
    
    disease_y_test.value_counts()
    disease_y_train.value_counts()
    preds = np.zeros(disease_y_test.shape[0])
    full_preds = np.zeros(full_y_test.shape[0])

    for fold, (idxT,idxV) in enumerate(skf.split(disease_train, disease_y_train)):

        X_train = disease_train.iloc[idxT]
        X_val = disease_train.iloc[idxV]
        y_train = disease_y_train.iloc[idxT]
        y_val = disease_y_train.iloc[idxV]
    
        XGB_model = XGBClassifier(n_estimators=5000, max_depth=None, 
                            learning_rate=0.05,
                            objective='binary:logistic', 
                            metric='auc',
                            verbosity  = VERBOSE,
                            # tree_method = 'gpu_hist',
                            n_jobs=-1, random_state  = SEED                            )
        
        if show_fold_stats:
            print('-' * 80)
            print('Fold : %s'%(fold+1))
    
        XGB_model.fit(X_train, y_train,
                        eval_set = [(X_val, y_val)],
                        eval_metric=['logloss'],
                        early_stopping_rounds = 100, verbose = VERBOSE )
            
        XGB_preds = XGB_model.predict_proba(X_val)
        XGB_score = metrics.roc_auc_score(y_val, XGB_preds[:,1])
        XGB_class = XGB_model.predict(X_val)
        
        XGB_test = XGB_model.predict_proba(disease_test)
        XGB_test_score = metrics.roc_auc_score(disease_y_test, XGB_test[:,1])
        XGB_test_class = XGB_model.predict(disease_test)
        
        full_test_preds = XGB_model.predict_proba(full_test)
        full_test_score = metrics.roc_auc_score(full_y_test, full_test_preds[:,1])
        full_test_class = XGB_model.predict(full_test)
               
        f1s = f1_score(y_val, XGB_class)
        recall = metrics.recall_score(y_val, XGB_class)
        precision_score = metrics.precision_score(y_val, XGB_class)
        
        f1_test = f1_score(disease_y_test, XGB_test_class)
        recall_test = metrics.recall_score(disease_y_test, XGB_test_class)
        precision_score_test = metrics.precision_score(disease_y_test, XGB_test_class)
        
        f1_full_test = f1_score(full_y_test, full_test_class)
        recall_full_test = metrics.recall_score(full_y_test, full_test_class)
        precision_full_test = metrics.precision_score(full_y_test, full_test_class)

        if show_fold_stats:        
            print('ROC AUC score for XGBoost model valid set: %.4f'%XGB_score)
            print('F1 score: %0.4f'%f1s)
            print(confusion_matrix(y_val, XGB_class))
    
            print('ROC AUC score for XGBoost model test set: %.4f'%XGB_test_score)
            print('F1 score: %0.4f'%f1_test)
            print(confusion_matrix(disease_y_test, XGB_test_class))
            
            print('ROC AUC score for full test set including other diseases used as false controls: %.4f'%full_test_score)
            print('F1 score: %0.4f'%f1_full_test)
            print(confusion_matrix(full_y_test, full_test_class))
        
        preds += XGB_test[:,1] / FOLDS
        full_preds += full_test_preds[:,1] / FOLDS
        
        fold_score = [XGB_score,f1s,recall,precision_score, XGB_test_score,f1_test,recall_test,precision_score_test]
        oof_scores.append({fold : fold_score})
    
    avg_test_score = metrics.roc_auc_score(disease_y_test, preds)
    avg_class = np.where(preds < 0.5, 0, 1)
    avg_f1_test = f1_score(disease_y_test, avg_class)
    avg_recall_test = metrics.recall_score(disease_y_test, avg_class)
    avg_precision_score_test = metrics.precision_score(disease_y_test, avg_class)

    avg_full_test_score = metrics.roc_auc_score(full_y_test, full_preds)
    avg_class_full = np.where(full_preds < 0.5, 0, 1)
    avg_f1_test_full = f1_score(full_y_test, avg_class_full)
    avg_recall_full_test = metrics.recall_score(full_y_test, avg_class_full)
    avg_precision_full_test = metrics.precision_score(full_y_test, avg_class_full)

    if show_fold_stats:        
        print('-' * 80)
        print('Confusion matrix for %s averaged across %i folds '%(d,FOLDS))
        print(confusion_matrix(disease_y_test, avg_class))

        print('ROC AUC score for %s averaged over %i folds: %.4f'%(d, FOLDS, avg_test_score))
        print('F1 : %.4f, Recall : %.4f , Precision : %.4f'%(avg_f1_test, avg_recall_test, avg_precision_score_test))
    
    print('ROC AUC score for %s against %s averaged over %i folds : %.4f'%(d, counter_d, FOLDS, avg_full_test_score ))
    print('F1 : %.4f, Recall : %.4f , Precision : %.4f'%(avg_f1_test_full, avg_recall_full_test, avg_precision_full_test))
    print('Confusion matrix for %s against %s averaged across %i folds'%(d, counter_d, FOLDS))
    print(confusion_matrix(full_y_test, avg_class_full))
    
    return full_preds, full_y_test, {d : (avg_test_score, avg_f1_test, avg_recall_test, avg_precision_score_test, oof_scores)}


def disease_prediction(d):
    print('-' * 80)
    print('Disease : %s'%d)
    
    skf = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = SEED)
    oof_preds = []
    oof_aucs = []
    oof_scores= []
    ds_names = data_sets[d]
    if len(ds_names) == 1:
        pd_cont = pd_control.loc[pd_control['dataset_name'] == ds_names[0]]
        pd_dis = pd_disease.loc[pd_disease['dataset_name'] == ds_names[0]]
    else:
        pd_cont = pd_control.loc[pd_control['dataset_name'] == ds_names[0]]
        pd_dis = pd_disease.loc[(pd_disease['disease'] == d) & (pd_disease['dataset_name'] == ds_names[0])]
        for ds in ds_names[1:]:
            pd_cont = pd.concat([pd_cont, pd_control.loc[pd_control['dataset_name'] == ds]], axis = 0)
            pd_dis = pd.concat([pd_dis, pd_disease.loc[(pd_disease['disease'] == d) & (pd_disease['dataset_name'] == ds)]], axis = 0)
                                 
    # create dataset with all other diseases, target set to 0
    pd_others = pd_disease.loc[(pd_disease['disease'] != d)]
    target_others = pd_others['disease'].apply(lambda x: 1 if x == d else 0)
    
    pd_train = pd.concat([pd_cont, pd_dis], axis = 0)
    # adding control data from healthy subject, data by HMP
    pd_train = pd.concat([pd_train, pd_control.loc[pd_control['dataset_name'] == 'hmp']], axis = 0)
    pd_train = pd.concat([pd_train, pd_control.loc[pd_control['dataset_name'] == 'hmpii']], axis = 0)
    target = pd_train['disease']
    # convert text target into binary
    binary_target = target.apply(lambda x: 1 if x == d else 0)
    # binary_target.value_counts()
    # use only abundance data for training
    pd_train  = pd_train[species] 
    disease_train, disease_test, disease_y_train, disease_y_test = train_test_split(pd_train, binary_target, test_size = 0.10, random_state = test_train_split_SEED)   
    pd_others = pd_others[species] 
    full_test = pd.concat([disease_test, pd_others])
    full_y_test = pd.concat([disease_y_test, target_others])
    
    disease_y_test.value_counts()
    disease_y_train.value_counts()
    preds = np.zeros(disease_y_test.shape[0])
    full_preds = np.zeros(full_y_test.shape[0])

    for fold, (idxT,idxV) in enumerate(skf.split(disease_train, disease_y_train)):

        X_train = disease_train.iloc[idxT]
        X_val = disease_train.iloc[idxV]
        y_train = disease_y_train.iloc[idxT]
        y_val = disease_y_train.iloc[idxV]
    
        XGB_model = XGBClassifier(n_estimators=5000, max_depth=None, 
                            learning_rate=0.05,
                            objective='binary:logistic', 
                            metric='auc',
                            verbosity  = VERBOSE,
                            # tree_method = 'gpu_hist',
                            n_jobs=-1, random_state  = SEED                            )
        
        if show_fold_stats:
            print('-' * 80)
            print('Fold : %s'%(fold+1))
    
        XGB_model.fit(X_train, y_train,
                        eval_set = [(X_val, y_val)],
                        eval_metric=['logloss'],
                        # eval_metric=['auc','logloss'],
                        early_stopping_rounds = 100, verbose = VERBOSE )
            
        XGB_preds = XGB_model.predict_proba(X_val)
        XGB_score = metrics.roc_auc_score(y_val, XGB_preds[:,1])
        XGB_class = XGB_model.predict(X_val)
        
        XGB_test = XGB_model.predict_proba(disease_test)
        XGB_test_score = metrics.roc_auc_score(disease_y_test, XGB_test[:,1])
        XGB_test_class = XGB_model.predict(disease_test)
        
        full_test_preds = XGB_model.predict_proba(full_test)
        full_test_score = metrics.roc_auc_score(full_y_test, full_test_preds[:,1])
        full_test_class = XGB_model.predict(full_test)
        
        # (fpr, tpr, thresholds) = metrics.roc_curve(y_val, XGB_preds[:,1])
        # plot_ROC(fpr, tpr, d)
        
        f1s = f1_score(y_val, XGB_class)
        recall = metrics.recall_score(y_val, XGB_class)
        precision_score = metrics.precision_score(y_val, XGB_class)
        
        f1_test = f1_score(disease_y_test, XGB_test_class)
        recall_test = metrics.recall_score(disease_y_test, XGB_test_class)
        precision_score_test = metrics.precision_score(disease_y_test, XGB_test_class)
        
        f1_full_test = f1_score(full_y_test, full_test_class)
        recall_full_test = metrics.recall_score(full_y_test, full_test_class)
        precision_full_test = metrics.precision_score(full_y_test, full_test_class)
        
        if show_fold_stats:        
            print('ROC AUC score for XGBoost model valid set: %.4f'%XGB_score)
            print('F1 score: %0.4f'%f1s)
            print(confusion_matrix(y_val, XGB_class))
    
            print('ROC AUC score for XGBoost model test set: %.4f'%XGB_test_score)
            print('F1 score: %0.4f'%f1_test)
            print(confusion_matrix(disease_y_test, XGB_test_class))
            
            print('ROC AUC score for full test set including other diseases used as false controls: %.4f'%full_test_score)
            print('F1 score: %0.4f'%f1_full_test)
            print(confusion_matrix(full_y_test, full_test_class))

        preds += XGB_test[:,1] / FOLDS
        full_preds += full_test_preds[:,1] / FOLDS
        
        fold_score = [XGB_score,f1s,recall,precision_score, XGB_test_score,f1_test,recall_test,precision_score_test]
        oof_scores.append({fold : fold_score})
    
    avg_test_score = metrics.roc_auc_score(disease_y_test, preds)
    avg_class = np.where(preds < 0.5, 0, 1)
    avg_f1_test = f1_score(disease_y_test, avg_class)
    avg_recall_test = metrics.recall_score(disease_y_test, avg_class)
    avg_precision_score_test = metrics.precision_score(disease_y_test, avg_class)

    avg_full_test_score = metrics.roc_auc_score(full_y_test, full_preds)
    avg_class_full = np.where(full_preds < 0.5, 0, 1)
    avg_f1_test_full = f1_score(full_y_test, avg_class_full)
    avg_recall_full_test = metrics.recall_score(full_y_test, avg_class_full)
    avg_precision_full_test = metrics.precision_score(full_y_test, avg_class_full)

    print('-' * 80)
    print('Confusion matrix for %s averaged across %i folds '%(d,FOLDS))
    print(confusion_matrix(disease_y_test, avg_class))

    print('Averaged over %i folds ROC AUC score for %s: %.4f'%(FOLDS,d,avg_test_score))
    print('F1 : %.4f, Recall : %.4f , Precision : %.4f'%(avg_f1_test, avg_recall_test, avg_precision_score_test))
    
    print('Averaged over %i folds ROC AUC score for %s against full set of other diseases: %.4f'%(FOLDS,d,avg_full_test_score))
    print('F1 : %.4f, Recall : %.4f , Precision : %.4f'%(avg_f1_test_full, avg_recall_full_test, avg_precision_full_test))
    print('Confusion matrix for %s averaged across %i folds for full set'%(d,FOLDS))
    print(confusion_matrix(full_y_test, avg_class_full))
    
    return preds, disease_y_test, full_preds, full_y_test
    # return preds, disease_y_test, {d : (avg_test_score, avg_f1_test, avg_recall_test, avg_precision_score_test, oof_scores)}, full_preds, full_y_test



In [26]:
SEEDs = [1969,1970,1971,1972,2020,2021]        
d = 'cirrhosis'
# SEED = 1970
d_preds = []
df_preds = []
dc_preds = []
for SEED in SEEDs:
    print('=' * 80)
    print('Seed : %i'%SEED)
    d_pred, y_true, full_preds, full_y_test = disease_prediction(d) 
    d_preds.append(d_pred)
    df_preds.append(full_preds)

#     for counter in diseases:
#         if counter != d:
#             dc_pred, y_preds, _ = disease_distinction(d, counter)

# print('-' * 80)

Seed : 1969
--------------------------------------------------------------------------------
Disease : cirrhosis
Averaged over 5 folds ROC AUC score for cirrhosis against full set of other diseases: 0.9915
F1 : 0.4471, Recall : 0.9500 , Precision : 0.2923
Confusion matrix for cirrhosis averaged across 5 folds for full set
[[815  46]
 [  1  19]]
Seed : 1970
--------------------------------------------------------------------------------
Disease : cirrhosis
Averaged over 5 folds ROC AUC score for cirrhosis against full set of other diseases: 0.9911
F1 : 0.4471, Recall : 0.9500 , Precision : 0.2923
Confusion matrix for cirrhosis averaged across 5 folds for full set
[[815  46]
 [  1  19]]
Seed : 1971
--------------------------------------------------------------------------------
Disease : cirrhosis
Averaged over 5 folds ROC AUC score for cirrhosis against full set of other diseases: 0.9917
F1 : 0.4872, Recall : 0.9500 , Precision : 0.3276
Confusion matrix for cirrhosis averaged across 5 f

In [27]:
print('Averaged predictions across %i seeds'%len(SEEDs))
avg_pred = np.array(d_preds).mean(axis= 0)
avg_class = np.where(avg_pred < 0.5, 0, 1)
print('Confusion matrix for %s'%d)
print(confusion_matrix(y_true, avg_class))

avg_pred = np.array(df_preds).mean(axis= 0)
avg_class = np.where(avg_pred < 0.5, 0, 1)
print('Confusion matrix for %s against controls and other diseases combined'%d)
print(confusion_matrix(full_y_test, avg_class))
print('F1 score : %.4f'%f1_score(full_y_test, avg_class))

Averaged predictions across 6 seeds
Confusion matrix for cirrhosis
[[23  0]
 [ 1 19]]
Confusion matrix for cirrhosis against controls and other diseases combined
[[819  42]
 [  1  19]]
F1 score : 0.4691


In [ ]:
dc_preds = []
for SEED in SEEDs:
    print('=' * 80)
    print('Seed : %i'%SEED)

    for counter in diseases:
        if counter != d:
            dc_pred, y_preds, _ = disease_distinction(d, counter)

    dc_preds.append(dc_pred)

Seed : 1969
--------------------------------------------------------------------------------
Discriminate cirrhosis from obesity
--------------------------------------------------------------------------------
Confusion matrix for cirrhosis averaged across 5 folds 
[[23  0]
 [ 1 19]]
ROC AUC score for cirrhosis averaged over 5 folds: 1.0000
F1 : 0.9744, Recall : 0.9500 , Precision : 1.0000
ROC AUC score for cirrhosis against obesity averaged over 5 folds : 0.9995
F1 : 0.9500, Recall : 0.9500 , Precision : 0.9500
Confusion matrix for cirrhosis against obesity averaged across 5 folds
[[186   1]
 [  1  19]]
--------------------------------------------------------------------------------
Discriminate cirrhosis from t2d
--------------------------------------------------------------------------------
Confusion matrix for cirrhosis averaged across 5 folds 
[[23  0]
 [ 1 19]]
ROC AUC score for cirrhosis averaged over 5 folds: 1.0000
F1 : 0.9744, Recall : 0.9500 , Precision : 1.0000
ROC AUC sco